<a href="https://colab.research.google.com/github/Alialtinel/aalto_summer24_project/blob/main/notebooks/filter_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
acces_token = 'access_token_here'
! git config --global user.email "alialtinel02@gmail.com"
! git config --global user.name "Alialtinel"

In [3]:
!git clone https://{acces_token}@github.com/Alialtinel/aalto_summer24_project.git

Cloning into 'aalto_summer24_project'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 29 (delta 4), reused 5 (delta 0), pack-reused 0
Receiving objects: 100% (29/29), 891.69 KiB | 3.96 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [4]:
%cd aalto_summer24_project
#!git pull

/content/aalto_summer24_project


In [5]:
df_2015 = pd.read_excel('/content/aalto_summer24_project/data/french_poverty_urban_areas_2015.xlsx')
df_2016 = pd.read_excel('/content/aalto_summer24_project/data/french_poverty_urban_areas_2016.xlsx')
df_2017 = pd.read_excel('/content/aalto_summer24_project/data/french_poverty_urban_areas_2017.xlsx')
column_mapping = pd.read_excel('/content/aalto_summer24_project/data/french_poverty_urban_areas_column_mapping.xlsx')
column_mapping

,column name,column code
0,Geographic Code,CODGEO
1,Geographic Label,LIBGEO
2,Number of Tax Households,NBMENFISC
3,Number of People in Tax Households,NBPERSMENFISC
4,Median Standard of Living (€),MED
5,Percentage of Taxable Households,PIMP
6,Overall Poverty Rate,TP60
7,Poverty Rate - Under 30 Years,TP60AGE1
8,Poverty Rate - 30 to 39 Years,TP60AGE2
9,Poverty Rate - 40 to 49 Years,TP60AGE3


In [6]:
common_numeric_columns = list(set(df_2015.select_dtypes(include=['int', 'float']).columns) &
                             set(df_2016.select_dtypes(include=['int', 'float']).columns) &
                             set(df_2017.select_dtypes(include=['int', 'float']).columns))

In [7]:
merged_df = df_2015.merge(df_2016, on='LIBGEO', suffixes=('_2015', '_2016')).merge(df_2017, on='LIBGEO', suffixes=('', '_2017'))

In [8]:
for col in common_numeric_columns:
  merged_df[col] = merged_df[[f"{col}_2015", f"{col}_2016", f"{col}"]].mean(axis=1)

# Drop the individual year columns
merged_df = merged_df.drop(columns=[f"{col}_2015" for col in common_numeric_columns] +
                           [f"{col}_2016" for col in common_numeric_columns] +
                           [f"{col}_2017" for col in common_numeric_columns if f"{col}_2017" in merged_df.columns])

merged_df = merged_df.drop(['CODGEO_2015', 'CODGEO_2016'] , axis = 1)


In [9]:
merged_df = merged_df.round(2)
merged_df

,LIBGEO,CODGEO,NBMENFISC,NBPERSMENFISC,MED,PIMP,TP60,TP60AGE1,TP60AGE2,TP60AGE3,...,PPEN,PPAT,PPSOC,PPFAM,PPMINI,PPLOGT,PIMPOT,D1,D9,RD
0,Paris,001,5064161.67,12386964.00,22911.38,64.53,15.33,19.07,16.30,17.57,...,21.60,10.87,4.40,1.90,1.37,1.13,-21.93,10580.38,46401.56,4.38
1,Lyon,002,939886.67,2231507.83,22157.60,58.20,12.87,19.67,14.57,15.00,...,24.40,10.53,5.17,2.30,1.50,1.33,-18.07,11502.95,40109.42,3.50
2,Marseille - Aix-en-Provence,003,734905.67,1705454.83,20635.56,53.13,18.63,27.37,21.70,21.97,...,27.80,10.37,6.57,2.20,2.53,1.80,-18.33,9878.33,38467.78,3.88
3,Toulouse,004,561315.67,1279804.83,22424.28,57.20,12.47,22.90,13.03,13.33,...,22.60,9.60,5.23,1.97,1.80,1.47,-18.03,11589.07,39787.93,3.43
4,Lille (partie française),005,471068.33,1125299.50,20269.67,51.40,18.07,27.80,19.73,20.57,...,23.43,8.93,7.30,2.70,2.63,1.97,-17.17,10212.01,37832.33,3.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
776,Saint-Paul,9D2,62601.00,180509.50,15537.56,32.57,35.37,46.67,36.73,37.47,...,13.77,7.00,14.20,4.17,7.40,2.63,-13.47,8045.70,37430.48,4.66
777,Saint-André,9D4,18881.33,57038.00,13145.89,20.40,46.07,62.87,50.73,47.93,...,11.23,5.70,22.10,6.43,10.60,5.10,-10.17,7426.92,29355.60,3.96
778,Saint-Louis,9D5,17838.00,53244.00,12777.20,19.87,48.40,64.03,52.13,51.37,...,14.20,5.63,22.80,6.47,11.83,4.50,-10.77,7251.29,27272.74,3.75
779,Saint-Joseph,9D6,14155.00,38220.50,13027.81,19.10,46.83,59.30,47.77,49.97,...,17.30,6.13,21.13,5.07,11.77,4.30,-9.70,6983.67,28255.60,4.05


In [10]:
merged_df = merged_df.set_index('LIBGEO')

In [11]:
import pandas as pd
df_french_cities = pd.read_excel('/content/aalto_summer24_project/data/french_cities.xlsx')
df_french_cities


,city_ascii,lat,lng
0,Paris,48.8567,2.3522
1,Bordeaux,44.8400,-0.5800
2,Marseille,43.2964,5.3700
3,Lyon,45.7600,4.8400
4,Toulouse,43.6045,1.4440
...,...,...,...
1147,Perenchies,50.6686,2.9731
1148,Thaon-les-Vosges,48.2510,6.4200
1149,Sautron,47.2644,-1.6667
1150,Liffre,48.2142,-1.5067


In [12]:
import re

matched_indices = {}

# Iterate over the cities in df_french_cities
for city in df_french_cities['city_ascii']:
  # Use regex to find close matches in the index of merged_df
  pattern = re.compile(rf"{city}(?: \(.+\))?")  # Allow optional parentheses with additional text
  matches = [idx for idx in merged_df.index if pattern.match(idx)]

  if matches:
    matched_indices[city] = matches
  else:
    matched_indices[city] = []  # No match found

# Print the matched indices
for city, indices in matched_indices.items():
  print(f"{city}: {indices}")


Paris: ['Paris']
Bordeaux: ['Bordeaux']
Marseille: ['Marseille - Aix-en-Provence']
Lyon: ['Lyon']
Toulouse: ['Toulouse']
Nice: ['Nice']
Nantes: ['Nantes']
Montpellier: ['Montpellier']
Strasbourg: ['Strasbourg (partie française)']
Lille: ['Lille (partie française)', 'Lillebonne']
Rennes: ['Rennes']
Toulon: ['Toulon']
Reims: ['Reims']
Saint-Etienne: []
Le Havre: ['Le Havre']
Dijon: ['Dijon']
Grenoble: ['Grenoble']
Angers: ['Angers']
Villeurbanne: []
Nimes: []
Aix-en-Provence: []
Clermont-Ferrand: ['Clermont-Ferrand']
Le Mans: ['Le Mans']
Brest: ['Brest']
Tours: ['Tours']
Amiens: ['Amiens']
Annecy: ['Annecy']
Limoges: ['Limoges']
Metz: ['Metz']
Boulogne-Billancourt: []
Perpignan: ['Perpignan']
Besancon: []
Orleans: []
Rouen: ['Rouen']
Saint-Denis: ['Saint-Denis']
Montreuil: ['Montreuil-Bellay']
Caen: ['Caen']
Argenteuil: []
Mulhouse: ['Mulhouse']
Nancy: ['Nancy']
Tourcoing: []
Roubaix: []
Nanterre: []
Vitry-sur-Seine: []
Creteil: []
Avignon: ['Avignon']
Poitiers: ['Poitiers']
Aubervillier

In [13]:
indices_to_keep = [idx for indices in matched_indices.values() for idx in indices]

# Filter the merged_df to include only these indices
filtered_df = merged_df.loc[indices_to_keep]

# Display the filtered DataFrame
filtered_df

,CODGEO,NBMENFISC,NBPERSMENFISC,MED,PIMP,TP60,TP60AGE1,TP60AGE2,TP60AGE3,TP60AGE4,...,PPEN,PPAT,PPSOC,PPFAM,PPMINI,PPLOGT,PIMPOT,D1,D9,RD
LIBGEO,,,,,,,,,,,,,,,,,,,,,
Paris,001,5064161.67,12386964.00,22911.38,64.53,15.33,19.07,16.30,17.57,16.00,...,21.60,10.87,4.40,1.90,1.37,1.13,-21.93,10580.38,46401.56,4.38
Bordeaux,006,527937.00,1196274.67,21900.44,56.13,12.03,22.30,13.37,13.10,11.50,...,26.27,9.37,4.97,1.83,1.77,1.37,-17.77,11760.29,38424.44,3.28
Marseille - Aix-en-Provence,003,734905.67,1705454.83,20635.56,53.13,18.63,27.37,21.70,21.97,19.23,...,27.80,10.37,6.57,2.20,2.53,1.80,-18.33,9878.33,38467.78,3.88
Lyon,002,939886.67,2231507.83,22157.60,58.20,12.87,19.67,14.57,15.00,12.37,...,24.40,10.53,5.17,2.30,1.50,1.33,-18.07,11502.95,40109.42,3.50
Toulouse,004,561315.67,1279804.83,22424.28,57.20,12.47,22.90,13.03,13.33,11.50,...,22.60,9.60,5.23,1.97,1.80,1.47,-18.03,11589.07,39787.93,3.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Aigues-Mortes,435,3915.67,8781.33,20073.27,49.90,15.33,NaN,17.13,16.57,18.13,...,32.87,13.57,5.13,1.80,1.97,1.33,-16.80,11040.67,34878.50,3.17
Hirson,367,4241.00,9122.50,15564.71,32.37,32.63,46.07,45.93,40.70,34.80,...,36.67,7.30,13.80,3.73,6.10,3.97,-11.83,8715.72,26889.56,3.08
Chamonix-Mont-Blanc,322,6483.00,13865.00,22078.48,60.17,12.77,NaN,14.93,15.13,13.57,...,24.07,18.43,2.37,1.23,0.60,0.57,-19.63,11361.57,43212.00,3.80


In [14]:
# Create a dictionary for efficient lookup of coordinates
city_coordinates = dict(zip(df_french_cities['city_ascii'], zip(df_french_cities['lat'], df_french_cities['lng'])))

# Add latitude and longitude columns to filtered_df
filtered_df['lat'] = filtered_df.index.map(lambda city: city_coordinates.get(city, (None, None))[0])
filtered_df['lng'] = filtered_df.index.map(lambda city: city_coordinates.get(city, (None, None))[1])

# Display the updated DataFrame
filtered_df


,CODGEO,NBMENFISC,NBPERSMENFISC,MED,PIMP,TP60,TP60AGE1,TP60AGE2,TP60AGE3,TP60AGE4,...,PPSOC,PPFAM,PPMINI,PPLOGT,PIMPOT,D1,D9,RD,lat,lng
LIBGEO,,,,,,,,,,,,,,,,,,,,,
Paris,001,5064161.67,12386964.00,22911.38,64.53,15.33,19.07,16.30,17.57,16.00,...,4.40,1.90,1.37,1.13,-21.93,10580.38,46401.56,4.38,48.8567,2.3522
Bordeaux,006,527937.00,1196274.67,21900.44,56.13,12.03,22.30,13.37,13.10,11.50,...,4.97,1.83,1.77,1.37,-17.77,11760.29,38424.44,3.28,44.8400,-0.5800
Marseille - Aix-en-Provence,003,734905.67,1705454.83,20635.56,53.13,18.63,27.37,21.70,21.97,19.23,...,6.57,2.20,2.53,1.80,-18.33,9878.33,38467.78,3.88,NaN,NaN
Lyon,002,939886.67,2231507.83,22157.60,58.20,12.87,19.67,14.57,15.00,12.37,...,5.17,2.30,1.50,1.33,-18.07,11502.95,40109.42,3.50,45.7600,4.8400
Toulouse,004,561315.67,1279804.83,22424.28,57.20,12.47,22.90,13.03,13.33,11.50,...,5.23,1.97,1.80,1.47,-18.03,11589.07,39787.93,3.43,43.6045,1.4440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Aigues-Mortes,435,3915.67,8781.33,20073.27,49.90,15.33,NaN,17.13,16.57,18.13,...,5.13,1.80,1.97,1.33,-16.80,11040.67,34878.50,3.17,43.5675,4.1933
Hirson,367,4241.00,9122.50,15564.71,32.37,32.63,46.07,45.93,40.70,34.80,...,13.80,3.73,6.10,3.97,-11.83,8715.72,26889.56,3.08,49.9217,4.0839
Chamonix-Mont-Blanc,322,6483.00,13865.00,22078.48,60.17,12.77,NaN,14.93,15.13,13.57,...,2.37,1.23,0.60,0.57,-19.63,11361.57,43212.00,3.80,45.9231,6.8697


In [15]:
# Save the DataFrame to a CSV file
%cd aalto_summer24_project
filtered_df.to_csv("data/filtered_data.csv")

# Add the file to the Git repository
!git add data/filtered_data.csv


# Commit the changes
!git commit -m "Add filtered data"

# Push the changes to the remote repository
!git push origin main


[Errno 2] No such file or directory: 'aalto_summer24_project'
/content/aalto_summer24_project
[main 6d57e44] Add filtered data
 1 file changed, 332 insertions(+)
 create mode 100644 data/filtered_data.csv
Enumerating objects: 6, done.
Counting objects: 100% (6/6), done.
Delta compression using up to 2 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 22.28 KiB | 2.79 MiB/s, done.
Total 4 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/Alialtinel/aalto_summer24_project.git
   ec7ec58..6d57e44  main -> main
